In [5]:
import openzwave
from openzwave.node import ZWaveNode
from openzwave.value import ZWaveValue
from openzwave.scene import ZWaveScene
from openzwave.controller import ZWaveController
from openzwave.network import ZWaveNetwork
from openzwave.option import ZWaveOption
import signal
import six
if six.PY3:
    from pydispatch import dispatcher
else:
    from louie import dispatcher

from datetime import datetime

import time
import json

device="/dev/ttyACM0"
#Define some manager options
options = ZWaveOption(device, \
  config_path="../config", \
  user_path=".", cmd_line="")
options.set_log_file("OZW_Log.log")
options.set_append_log_file(False)
options.set_console_output(True)
# options.set_save_log_level(log)
options.set_save_log_level('Info')
options.set_logging(False)
options.lock()

#Create a network object
network = ZWaveNetwork(options, log=None)
# node = man.addNode(homeId)
# watch = man.addWatcher(cb)
# driver = man.addDriver(device)

exit = False
man = network.manager
# node = man.addNode(homeId)
# watch = man.addWatcher(cb)
# driver = man.addDriver(device)

print('setup complete')

setup complete


In [4]:
exit = False
def sigint_handler(signal, frame):
    global exit
    exit = True
signal.signal(signal.SIGINT, sigint_handler)

data = []
watts = []
kwatth = []
current = []


# Connect to events
def value_updated(network, node, value):
    # data.append([value.parent_id, value.label, value.data_as_string, value.units])

    # if value.units == 'kWh':
    #     kwatth.append([value.parent_id, value.data])
    # if value.units == 'W':
    #     watts.append([value.parent_id, value.data])
    # if value.units == 'A':
    #     current.append([value.parent_id, value.data])
    
    # print(value.parent_id, value.label, value.data, value.units, value.command_class) 
    pass
    
dispatcher.connect(value_updated, ZWaveNetwork.SIGNAL_VALUE)

# def value_updated(uhh):
#     print(uhh) 
# dispatcher.connect(value_updated, openzwave.network)

value_ids = {}

for node in network.nodes:
    print("node number:", node)
    sensor_dat = network.nodes[node].get_configs()
    for key, value in sensor_dat.items():
        # sensor_dat[key]
        print(key, value, value.value_id)
        if value.value_id in value_ids:
            value_ids[value.value_id] += 1
        else:
            value_ids[value.value_id] = 1
        # for thang in thing
        #     print(thang)

# Loop until we're told to exit:
print("Running: Ctrl+C to exit.")

# log stuff
filetime = "value_ids_" + time.strftime("%Y%m%d-%H%M%S")



counter = 0

while not exit:
    # df.to_csv(csv)
    # openzwave.comm
    for node in network.nodes:
        print("node number:", node)
        sensor_dat = network.nodes[node].get_sensors()
        for key, value in sensor_dat.items():
            # sensor_dat[key]
            # print(key, value, value.value_id)
            if value.value_id in value_ids:
                value_ids[value.value_id] += 1
                print(network.nodes[node].get_sensor_value(value.value_id))
            else:
                value_ids[value.value_id] = 1    


    if (counter % 10) == 0:
        #log = open(filetime, "w")
        #log.write(json.dumps(value_ids))
        #log.close()

    

    # print(value_ids)
    time.sleep(3)
    counter+=1
    
    if len(value_ids)==84:
        exit()
    # signal.pause()
dispatcher.disconnect(value_updated, ZWaveNetwork.SIGNAL_VALUE)



Running: Ctrl+C to exit.


In [ ]:
import matplotlib.pyplot as plt

# organize data

unique_ids = []
for id in data:
    if id[0] not in unique_ids:
        unique_ids.append(id[0])

current_per_id = dict.fromkeys(unique_ids, [])
watts_per_id = dict.fromkeys(unique_ids, [])
kwh_per_id = dict.fromkeys(unique_ids, [])

for w in watts:
    watts_per_id[w[0]].append(w[1])

for c in current:
    current_per_id[c[0]].append(c[1])

for kwh in kwatth:
    kwh_per_id[kwh[0]].append(c[1])


meas = [x for x in range(30)]

for key in watts_per_id:
    plt.plot(meas, watts_per_id[key], label=key)
plt.show()

READ MORE
https://pyozw.readthedocs.io/en/latest/command.html

AND 
https://www.silabs.com/documents/public/application-notes/APL12955-Z-Wave-Multi-Channel-Basics.pdf